In [ ]:
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
import plotly.express as px
import scipy.optimize


"Начальные значения"


birth : float = np.array([2,1])
death : float = np.array([1,2])
comp : float = np.array([[1,2],[2,1]])


"Переход из двумерного в одномерное представления состояния"


def from2dto1d(v):
  return int(v[1]*(v[1]-1)*0.5 + (v[0]-1)*(v[0]*0.5+v[1]))

def from1dto2d(k):
  i = 1
  while (int(i*(i+1)*0.5) - 1 < k): #находим номер диагонали
    i+=1
  return (k - int(i*(i-1)*0.5) + 1, int(i*(i+1)*0.5) - k)


"Расчет среднего количества особей"


def que(v, n):
  sum_1 = np.zeros(n)
  sum_2 = np.zeros(n)
  for i in range(n):
    sum_1[from1dto2d(i)[0]] += v[i]
    sum_2[from1dto2d(i)[1]] += v[i]
  que1 = 0
  que2 = 0
  for i in range(n):
    que1 += sum_1[i]*i
    que2 += sum_2[i]*i
  return que1, que2

def que2d(v, n):
  sum_1 = np.zeros(n)
  sum_2 = np.zeros(n)
  for i in range(n):
    for j in range(n):
      sum_1[i] += v[i][j]
      sum_2[j] += v[i][j]
  que1 = 0
  que2 = 0
  for i in range(n):
    que1 += sum_1[i]*i
    que2 += sum_2[i]*i
  return que1, que2

def que_dif(que1, que2):
  n = len(que1) - 1
  que1_diff = np.zeros(n)
  que2_diff = np.zeros(n)
  for i in range(n):
    que1_diff[i] = que1[i+1]-que1[i]
    que2_diff[i] = que2[i+1]-que2[i]
  return que1_diff, que2_diff

"Вычисление максимальной абсолютной разницы между частотами текущей некого предыдущего состояния"

def v_error(v, v_prev, eps):
  sz = len(v)
  if_close = np.zeros((sz, sz))
  sum_part_v = 0
  for i in range(sz):
    for j in range(sz):
      if v[i][j] == 0:
        if_close[i][j] = -1
      elif abs(v[i][j] - v_prev[i][j]) < eps:
        if_close[i][j] = 1
        sum_part_v += v[i][j]
      else:
        if_close[i][j] = 0
  return sum_part_v, if_close

"Вычисление характеристик относительно 'хороших' состояний" #NOT FINISHED; WHAT IS 'хорошее' состояние?#

def que_part(v, if_state):
  sum_1 = np.zeros(len(v))
  sum_2 = np.zeros(len(v))
  for i in range(len(v)):
    for j in range(len(v)):
      if (if_state[i][j] == 1):
        sum_1[i] += v[i][j]
        sum_2[j] += v[i][j]
  que1 = 0
  que2 = 0
  for i in range(len(v)):
    que1 += sum_1[i]*i
    que2 += sum_2[i]*i
  return que1, que2


"Переход в следующее состояние системы"


"""
dictionary for states
0 - down(b0) 1 - right(b1) 2 - up(d0 c00 c01) 3 - left(d1 c10 c11)
"""
direction_to_next_state = {
    0 : [1,0],
    1 : [0,1],
    2 : [-1,0],
    3 : [0,-1]
}

def next_chosen_state(state,b,c,d, V_inv):
  """
  evaluating the probabilities
  """
  lambda_sum : float = 0
  next_state_prob : float = np.zeros(4)
  next_state_prob[0] = b[0]*state[0] * V_inv
  next_state_prob[1] = b[1]*state[1] * V_inv
  lambda_sum += next_state_prob[0] + next_state_prob[1]
  if (state[0] > 1):
    next_state_prob[2] = d[0]*state[0] * V_inv + (c[0][0]*pow(state[0],2) + c[0][1]*state[0]*state[1]) * V_inv * V_inv
    lambda_sum += next_state_prob[2]
  if (state[1] > 1):
    next_state_prob[3] = d[1]*state[1] * V_inv + (c[1][1]*pow(state[1],2) + c[1][0]*state[1]*state[0]) * V_inv * V_inv
    lambda_sum += next_state_prob[3]
  n_s_p = [x / lambda_sum for x in next_state_prob]
  """
  choosing the direction (left up right down); next state
  """
  x = np.random.uniform(0,1)
  i : int = -1
  while (x > 0):
    i += 1
    x -= n_s_p[i]

  state[0] += direction_to_next_state[i][0]
  state[1] += direction_to_next_state[i][1]

  """
  evaluating time of transition into next state
  """
  t = np.random.exponential(1/sum(next_state_prob))

  return t, state

def next_state(state, TDeg, SDeg, N1, N2, b, c, d, V_inv):
  t, state = next_chosen_state(state, b, c, d, V_inv)
  TDeg += t
  SDeg += 1
  N1 += state[0]
  N2 += state[1]
  return state, TDeg, SDeg, N1, N2

"Вариация параметров"

def generation_param(code, fst_spc, char_type, i, N = 100):
  match code:
    case 'b1':
      return generation([birth[0]+0.05*i, birth[1]], comp, death, fst_spc, N)[char_type]
    case 'b2':
      return generation([birth[0], birth[1]+0.05*i], comp, death, fst_spc, N)[char_type]
    case 'c11':
      return generation(birth, [[comp[0][0]+0.05*i,comp[0][1]], [comp[1][0], comp[1][1]]], death, fst_spc, N)[char_type]
    case 'c12':
      return generation(birth, [[comp[0][0],comp[0][1]+0.05*i], [comp[1][0], comp[1][1]]], death, fst_spc, N)[char_type]
    case 'c21':
      return generation(birth, [[comp[0][0],comp[0][1]], [comp[1][0]+0.05*i, comp[1][1]]], death, fst_spc, N)[char_type]
    case 'c22':
      return generation(birth, [[comp[0][0],comp[0][1]], [comp[1][0], comp[1][1]+0.05*i]], death, fst_spc, N)[char_type]
    case 'd1':
      return generation(birth, comp, [death[0]+0.05*i, death[1]], fst_spc, N)[char_type]
    case 'd2':
      return generation(birth, comp, [death[0], death[1]+0.05*i], fst_spc, N)[char_type]
    case _:
      raise Exception("Wrong parameter code")

Генерации

In [ ]:
"Классический вариант симуляции"

def generation(b,c,d,fst_spc = 1, scd_spc = 1, N = 100, V = 5):
  tdeg : float = 0
  sdeg : float = 0
  mn1 : float = 0
  mn2 : float = 0
  N_inv : float = 1 / N
  V_inv : float = 1 / V

  for i in range(N):
    i_state : float = np.array([fst_spc, scd_spc])
    i_time : float = 0
    i_s : float = 0
    i_n1 : float = fst_spc
    i_n2 : float = scd_spc
    while(i_state[0] + i_state[1] != 0):
      i_state, i_time, i_s, i_n1, i_n2 = next_state(i_state, i_time, i_s, i_n1, i_n2, b, c, d, V_inv)
      "print(i_s, i_state, i_time)"
    i_n1 /= i_s
    i_n2 /= i_s
    tdeg += i_time
    sdeg += i_s
    mn1 += i_n1
    mn2 += i_n2
  tdeg *= N_inv
  sdeg *= N_inv
  mn1 *= N_inv
  mn2 *= N_inv

  return tdeg, sdeg, mn1, mn2

def single_generation(b,c,d,fst_spc = 1, scd_spc = 1, V = 5):
  return generation(b, c, d, fst_spc, scd_spc, 1, V)

"Генерации с вариацией параметра"

def generation_variation(param_code, fst_spc, char_type, N):
  x = np.linspace(start=2, stop=5, num=61, endpoint=True)
  y = np.zeros(61)
  match char_type:
    case 'tdeg':
      char : int = 0
    case 'sdeg':
      char : int = 1
    case 'mn1':
      char : int = 2
    case 'mn2':
      char : int = 3
    case _:
      raise Exception("Wrong parameter code")
  for i in range(61):
    y[i] : float = generation_param(param_code, fst_spc, char, i, N)
  return x, y

"Генерация с ограниченным количеством шагов"

def finite_generation(b,c,d,number_of_steps = 1000, fst_spc = 1, scd_spc = 1, N = 100, bound = 5500):
  tdeg : float = 0
  sdeg : float = 0
  mn1 : float = 0
  mn2 : float = 0
  n1 : int = 0
  n2 : int = 0
  maxn1 : int = 1
  maxn2 : int = 1
  N_inv : float = 1 / N
  sz = min(bound, int(number_of_steps * (number_of_steps + 1) / 2))
  Vf = np.zeros(sz)

  for i in range(N):
    i_state : int = np.array([fst_spc, scd_spc])
    i_time : float = 0
    i_s : float = 0
    i_n1 : float = fst_spc
    i_n2 : float = scd_spc
    j = 0
    while(i_state[0] + i_state[1] != 0 and j < number_of_steps):
      j += 1
      Vf[from2dto1d(i_state)] += 1
      i_state, i_time, i_s, i_n1, i_n2 = next_state(i_state, i_time, i_s, i_n1, i_n2, b, c, d, 1.0 / (i_n1 + i_n2))
      if i_state[0] > maxn1:
        maxn1 = i_state[0]
      if i_state[1] > maxn2:
        maxn2 = i_state[1]
      "print(i_s, i_state, i_time)"
    i_n1 /= i_s
    i_n2 /= i_s
    tdeg += i_time
    sdeg += i_s
    mn1 += i_n1
    mn2 += i_n2
    n1 = i_state[0]
    n2 = i_state[1]
  tdeg *= N_inv
  sdeg *= N_inv
  mn1 *= N_inv
  mn2 *= N_inv
  n1 *= N_inv
  n2 *= N_inv
  Vf *= N_inv / number_of_steps

  return n1, n2, tdeg, sdeg, mn1, mn2, Vf, maxn1, maxn2

"Генерация пока не достигнуто квазистационарное состояние системы" #NOT FINISHED#

def generation_until_error(b,c,d, interval = 1000, fst_spc = 1, scd_spc = 1, num_of_intervals = 100, bound = 100, eps = 0.001, sum_stab_v = 0.9):
  i_state : int = np.array([fst_spc, scd_spc])
  i_time : float = 0
  i_s : float = 0
  i_n1 : float = fst_spc
  i_n2 : float = scd_spc
  sz = min(bound, interval*num_of_intervals)
  Vf = np.zeros((sz, sz))
  v = np.zeros((sz, sz))
  v_prev = np.zeros((sz, sz))
  if_close = np.zeros((sz, sz))
  sum_part_v = 0

  k = 0
  while (sum_part_v <= sum_stab_v and k < num_of_intervals):
    v_prev = v
    for i in range(interval*k, interval*(k+1)):
      Vf[i_state[0]][i_state[1]] += 1
      i_state, i_time, i_s, i_n1, i_n2 = next_state(i_state, i_time, i_s, i_n1, i_n2, b, c, d, 1.0 / (i_n1 + i_n2))
    v = Vf / (interval*(k+1))
    sum_part_v, if_close = v_error(v, v_prev, eps)
    print(sum_part_v)
    k += 1

  return sum_part_v, k, que_part(v, if_close), v, v_prev, if_close

"Генерация с ограниченным количеством шагов и последовательным выведением характеристик"

#DEPRECATED#
def finite_generation_interval_ques(b,c,d,number_of_steps = 1000, char_intervals = 10, fst_spc = 1, scd_spc = 1):
  i_state : int = np.array([fst_spc, scd_spc])
  i_time : float = 0
  i_s : float = 0
  i_n1 : float = fst_spc
  i_n2 : float = scd_spc
  max_state : int = int(number_of_steps * (number_of_steps + 1) / 2)
  interval = int(number_of_steps / char_intervals)
  Vf = np.zeros(max_state)

  que1 = np.zeros(char_intervals)
  que2 = np.zeros(char_intervals)

  for k in range(char_intervals):
    for i in range(interval*k, interval*(k+1)):
      Vf[from2dto1d(i_state)] += 1
      i_state, i_time, i_s, i_n1, i_n2 = next_state(i_state, i_time, i_s, i_n1, i_n2, b, c, d, 1.0 / (i_n1 + i_n2))
    v = Vf / (interval*(k+1))
    que1[k], que2[k] = que(v, int(interval*(k+1) * (interval*(k+1) + 1) * 0.5))

  return que1, que2

#USE THIS INSTEAD#
def finite_generation_ques(b,c,d,number_of_steps = 1000, char_intervals = 10, fst_spc = 1, scd_spc = 1, bound = 100):
  i_state : int = np.array([fst_spc, scd_spc])
  i_time : float = 0
  i_s : float = 0
  i_n1 : float = fst_spc
  i_n2 : float = scd_spc
  interval = int(number_of_steps / char_intervals)
  sz = min(bound, number_of_steps*char_intervals)
  Vf = np.zeros((sz, sz))

  que1 = np.zeros(char_intervals)
  que2 = np.zeros(char_intervals)

  for k in range(char_intervals):
    for i in range(interval*k, interval*(k+1)):
      Vf[i_state[0]][i_state[1]] += 1
      i_state, i_time, i_s, i_n1, i_n2 = next_state(i_state, i_time, i_s, i_n1, i_n2, b, c, d, 1.0 / (i_n1 + i_n2))
    v = Vf / (interval*(k+1))
    que1[k], que2[k] = que2d(v, sz)
    #print("que1_k {} que1_k-1 {} que2_k {} que2_k-1 {}".format(round(que1[k],5),round(que1[k-1],5),round(que2[k],5),round(que2[k-1],5)))

  return que1, que2

"Генерация с ограниченным количеством шагов и последовательным выведением характеристик с N траекториями"

def finite_generation_ques_N_generations(b,c,d,number_of_steps = 1000, char_intervals = 10, fst_spc = 1, scd_spc = 1, bound = 100, N = 10):
  N_que1, N_que2 = np.zeros(char_intervals), np.zeros(char_intervals)
  for i in range(N):
    print("iteration={}".format(i))
    tmp_q1, tmp_q2 = finite_generation_ques(b,c,d,number_of_steps,char_intervals,fst_spc,scd_spc,bound)
    for j in range(char_intervals):
      N_que1[j] += tmp_q1[j]
      N_que2[j] += tmp_q2[j]
  return N_que1 / N, N_que2 / N

"Генерация пока que_i имеют разность больше eps"

def generation_ques_until_error(b,c,d, interval = 1000, fst_spc = 1, scd_spc = 1, num_of_intervals = 100, bound = 100, eps=0.0001):
  i_state : int = np.array([fst_spc, scd_spc])
  i_time : float = 0
  i_s : float = 0
  i_n1 : float = fst_spc
  i_n2 : float = scd_spc
  Vf = np.zeros((bound, bound))

  que1 = np.zeros(num_of_intervals)
  que2 = np.zeros(num_of_intervals)

  k = 0
  for i in range(interval):
      Vf[i_state[0]][i_state[1]] += 1
      i_state, i_time, i_s, i_n1, i_n2 = next_state(i_state, i_time, i_s, i_n1, i_n2, b, c, d, 1.0 / (i_n1 + i_n2))
  v = Vf / interval
  que1[0], que2[0] = que2d(v, bound)
  for i in range(interval):
      Vf[i_state[0]][i_state[1]] += 1
      i_state, i_time, i_s, i_n1, i_n2 = next_state(i_state, i_time, i_s, i_n1, i_n2, b, c, d, 1.0 / (i_n1 + i_n2))
  v = Vf / (interval*2)
  que1[1], que2[1] = que2d(v, bound)
  k = 2
  while (abs(que1[k-1] - que1[k-2]) > eps and abs(que2[k-1] - que2[k-2]) > eps and k < num_of_intervals):
    for i in range(interval*k, interval*(k+1)):
      Vf[i_state[0]][i_state[1]] += 1
      i_state, i_time, i_s, i_n1, i_n2 = next_state(i_state, i_time, i_s, i_n1, i_n2, b, c, d, 1.0 / (i_n1 + i_n2))
    v = Vf / (interval*(k+1))
    que1[k], que2[k] = que2d(v, bound)
    print("que1_k {} que1_k-1 {} que2_k {} que2_k-1 {}".format(round(que1[k],5),round(que1[k-1],5),round(que2[k],5),round(que2[k-1],5)))
    k += 1

  return k, que1, que2

Построение графиков

In [6]:
"Построение зависимости характеристик от вариации параметра" #6 sem

def plot_dependency(param_code, fst_spc, char_type = 'tdeg', plot_color = 'r', N = 100):
  x, y = generation_variation(param_code, fst_spc, char_type, N)
  fig = plt.figure(param_code + ' (' + str(fst_spc) + ',' + str(5-fst_spc) + '); ' + char_type)
  plt.plot(x, y, linewidth=1, color=plot_color)
  plt.title('Varying of parameter ' + param_code + ' starting from (' + str(fst_spc) + ',' + str(5-fst_spc) + ') with ' + str(N) + ' trajectories')
  plt.ylabel(char_type)

"Построение характеристик que_i"

def plot_ques(number_of_steps, char_intervals, que1, que2, y_lim=[-1,-1]):
  if y_lim ==[-1,-1]:
    y_lim = [0, max(max(que1),max(que2))]
  h = int(number_of_steps/char_intervals)
  x = np.arange(h, number_of_steps + h, h)
  plt.plot(x, que1, label = "que1", color = 'r')
  plt.plot(x, que2, label = "que2", color = 'b')
  plt.ylim(y_lim)
  plt.legend()

"Построение характеристик que_i и наложение функции методом МНК"

def plot_ques_and_fit(f, number_of_steps, char_intervals, que1, que2, plot_label):
  h = int(number_of_steps/char_intervals)
  x = np.arange(h, number_of_steps + h, h)

  fig = plt.figure("que1")
  plt.plot(x, que1, label = "que1", color = 'r')
  plt.title("que1")
  a, b = scipy.optimize.curve_fit(f, xdata = x, ydata = que1)[0]
  plt.plot(x,f(x,a,b), label = plot_label, color = 'g', linestyle='-.')
  plt.legend()

  fig = plt.figure("que2")
  plt.plot(x, que2, label = "que2", color = 'b')
  plt.title("que2")
  a, b = scipy.optimize.curve_fit(f, xdata = x, ydata = que2)[0]
  plt.plot(x,f(x,a,b), label = plot_label, color = 'g', linestyle='-.')
  plt.legend()

"Сохранить que_i как таблицу"

def save_ques_as_table(que1, que2):
  df = pd.DataFrame(data={"que1" : que1, "que2" : que2})
  df.to_excel("ques.xlsx")

Рабочая часть